### Load packages
First we need to load the python packages that are required to perform the analysis

In [1]:
import os
import rasterio
import numpy 
import pandas
import scipy.io # to load the matlab files

from DamageScanner import DamageScanner # the tool to estimate the losses

data_path = os.path.join('..','Data')

### Set paths to data and load initial data

All the data is stored in the *Data* folder, so we are creating paths to load data from that folder. We will create a dictionary with the depth-damage information required to do the analysis.

In [2]:
# function to load DDM data
def read_DDM_input(DDM_data_path):
    DDM_data = {}
    DDM_data['Curves_structure'] = pandas.read_excel(DDM_data_path,sheet_name='curves_structure').values
    DDM_data['Curves_content'] = pandas.read_excel(DDM_data_path,sheet_name='curves_content').values
    DDM_data['MaxDam_structure'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_structure').values
    DDM_data['MaxDam_content'] = pandas.read_excel(DDM_data_path,sheet_name='maxdam_content').values  
    
    return DDM_data

In [3]:
inundation_maps = scipy.io.loadmat(os.path.join(data_path,'Waterdepth_current.mat')) # the inundation files for the current situation
landuse_map = os.path.join(data_path,'landuse.asc') # the land-use map
DDM_data = read_DDM_input(os.path.join(data_path,'DDM_data_AUC.xlsx'))

### Specify for which return period we want to estimate damages

The return periods are 1/10,1/100,1/1000,1/2000,1/4000,1/10000

In [16]:
return_period = 2000

### Run the DamageScanner. We run it for both the structure and the content

In [17]:
# run the DamageScanner for the building structures
damage_structure,damagebin_structure = DamageScanner(landuse_map,inundation_maps['a0_{}'.format(return_period)],DDM_data['Curves_structure'],
                                                     DDM_data['MaxDam_structure'])

# run the DamageScanner for the content
damage_content,damagebin_content = DamageScanner(landuse_map,inundation_maps['a0_{}'.format(return_period)],DDM_data['Curves_content'],
                                                     DDM_data['MaxDam_content'])
# summarize both
damage_total = damage_content+damage_structure
damagebin_total  = pandas.DataFrame(pandas.concat([pandas.DataFrame(damagebin_structure['losses']), pandas.DataFrame(damagebin_content['losses'])], axis=1).fillna(0).sum(axis=1),columns=['losses'])
damagebin_total = pandas.concat([damagebin_total,damagebin_structure[['area','avg_depth']]],axis=1)

#print output
print('{} Euro damage for a 1/{} flood event'.format(damage_total,return_period))

31510441.289999977 Euro damage for a 1/2000 flood event


In [12]:
damagebin_total

,losses,area,avg_depth
landuse,,,
110.0,1.231764e+06,3751.0,26.979472
111.0,8.952698e+06,995.0,26.446231
112.0,4.773563e+05,144.0,27.597222
120.0,0.000000e+00,0.0,0.000000
130.0,2.441414e+05,739.0,51.774019
131.0,1.822791e+06,152.0,42.309211
132.0,0.000000e+00,0.0,0.000000
133.0,0.000000e+00,0.0,0.000000
134.0,5.504100e+03,2.0,3.500000
